In [640]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from pyzipcode import ZipCodeDatabase

In [641]:
class RoofImóveis():
    """
        A empresa te contratou pois quer saber qual seria os 5 imóveis que
        ela deveria investir e o porquê e quais 5 você não recomendaria o
        investimento de jeito nenhum.
    """

    def __init__(self):
        path = r'data/kc_house_data.csv'
        
        self.df = pd.read_csv(
            filepath_or_buffer=path,
            date_parser=['date'],
            sep=',',
            decimal='.',
        )

        self.df['date'] = pd.to_datetime(
            arg=self.df['date'],
            format='%Y%m%dT%H%M%S'
        )

        #Removendo ids duplicados
        self.df = self.df.sort_values(
            by='date',
            ascending=False
        ).drop_duplicates(subset=['id'])

        # Transformando a coluna bathrooms em inteiro:
        self.df['bathrooms'] = self.df['bathrooms'].astype(int)

        # Transformando a coluna floors em inteiro:
        self.df['floors'] = self.df['floors'].astype(int)

        # Criando a coluna cidade:
        zcdb = ZipCodeDatabase()
        city = []

        for zipcode in self.df['zipcode']:
            try:
                zc = zcdb[zipcode]
                city.append(zc.city)
                
            except KeyError:
                if zipcode == 98077:
                    city.append(str('Woodinville'))
                else:
                    city.append(str('nan'))
        
        self.df['city'] = city

    def média_do_valor_do_ft2_por_cidade(self):
        #Preciso elimina outliers
        df = self.df[
            [
                'city',
                'price',
                'sqft_lot',
            ]
        ]

        self.df['price_of_ft2'] = df['price_of_ft2_per_city'] = df['price'] / df['sqft_lot']

        df = df[(np.abs(stats.zscore(df['price_of_ft2_per_city'])) < 3)]
        df = df.groupby('city').agg({'price_of_ft2_per_city': 'mean'}).reset_index()

        price_of_ft2_per_city = []

        for city in self.df['city']:
            pompc = df[
                (df['city'] == city)
            ]['price_of_ft2_per_city'].iloc[0]

            price_of_ft2_per_city.append(pompc)

        self.df['price_of_ft2_per_city'] = price_of_ft2_per_city

    def média_do_tamanho_do_terreno_por_cidade(self):
        df = self.df[
            [
                'city',
                'sqft_lot',
            ]
        ]

        df = df.groupby('city').agg({'sqft_lot': 'median'}).reset_index()

        plot_size_by_city = []

        for city in self.df['city']:
            sqft_lot = df[
                (df['city'] == city)
            ]['sqft_lot'].iloc[0]

            plot_size_by_city.append(sqft_lot)

        self.df['plot_size_by_city'] = plot_size_by_city

    def média_de_quantidade_quartos_por_cidade(self):
        df = self.df[
            [
                'city',
                'bedrooms',
            ]
        ]

        df = df.groupby('city').agg({'bedrooms': 'median'}).astype(int).reset_index()

        bedrooms_by_city = []

        for city in self.df['city']:
            rbc = df[
                (df['city'] == city)
            ]['bedrooms'].iloc[0]

            bedrooms_by_city.append(rbc)

        self.df['bedrooms_by_city'] = bedrooms_by_city

    def média_de_quantidade_banheiros_por_cidade(self):
        df = self.df[
            [
                'city',
                'bathrooms',
            ]
        ]

        df = df.groupby('city').agg({'bathrooms': 'median'}).astype(int).reset_index()

        bathrooms_by_city = []

        for city in self.df['city']:
            bbc = df[
                (df['city'] == city)
            ]['bathrooms'].iloc[0]

            bathrooms_by_city.append(bbc)

        self.df['bathrooms_by_city'] = bathrooms_by_city

    def filtrando_os_melhores_imóveis(
        self,
        condition_min=5,
        yr_built_min=1980,
        bedrooms_min=1,
        bathrooms_min=1,
        price_max=1000000.0,
    ):

        # self.média_do_valor_do_ft2_por_cidade()

        df = self.df[
            (self.df['price'] / self.df['sqft_lot'] < self.df['price_of_ft2_per_city']) &
            (self.df['condition'] == condition_min) &
            (self.df['yr_built'] >= yr_built_min) &
            (self.df['bedrooms'] >= bedrooms_min) &
            (self.df['bathrooms'] >= bathrooms_min) &
            (self.df['price'] <= price_max)
        ].sort_values(by='price').reset_index()

        self.imóveis_filtrados = df

    def algoritmo_de_rankeamento_de_imóveis(self):
        # self.média_do_tamanho_do_terreno_por_cidade()
        # self.média_de_quantidade_quartos_por_cidade()
        # self.média_de_quantidade_banheiros_por_cidade()
        # self.filtrando_os_melhores_imóveis()

        self.rank_dos_imóveis = {}
        cidades_mais_populosas = [
            'Seattle', 
            'Spokane',
            'Tacoma',
            'Vancouver',
            'Bellevue',
            'Kent',
            'Everett',
            'Renton',
            'Federal Way',
            'Spokane Valley',
        ]

        for id in self.imóveis_filtrados['id']:
            self.rank_dos_imóveis[id] = 0

            city = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['city'].iloc[0]

            price_of_ft2 = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['price_of_ft2'].iloc[0]

            price_of_ft2_per_city = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['price_of_ft2_per_city'].iloc[0]

            sqft_lot = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['sqft_lot'].iloc[0]

            plot_size_by_city = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['plot_size_by_city'].iloc[0]

            bedrooms = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['bedrooms'].iloc[0]

            bedrooms_by_city = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['bedrooms_by_city'].iloc[0]

            bathrooms = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['bathrooms'].iloc[0]

            bathrooms_by_city = self.imóveis_filtrados[
                (self.imóveis_filtrados['id'] == id)
            ]['bathrooms_by_city'].iloc[0]


            resultado = (price_of_ft2_per_city - price_of_ft2) / price_of_ft2 * 100

            if city in cidades_mais_populosas:
                self.rank_dos_imóveis[id] += 30

            if resultado > 50:
                self.rank_dos_imóveis[id] += 30

            if sqft_lot > plot_size_by_city:
                self.rank_dos_imóveis[id] += 20 

            if bedrooms > bedrooms_by_city:
                self.rank_dos_imóveis[id] += 10

            if bathrooms > bathrooms_by_city:
                self.rank_dos_imóveis[id] += 10

        self.rank_dos_imóveis

    def os_5_melhores_imóveis(self):

        notas_ordenadas = sorted(self.rank_dos_imóveis.values(), reverse=True)
        cinco_maiores_notas = notas_ordenadas[:5]

        self.ids = [k for k, v in self.rank_dos_imóveis.items() if v in cinco_maiores_notas]

        df = self.df[
            (self.df['id'].isin(self.ids))
        ].sort_values(by='price').reset_index()

        display(
            df[
                [
                    'id',
                    'city',
                    'price',
                    'sqft_lot',
                    'price_of_ft2',
                    'price_of_ft2_per_city',
                    'condition',
                    'yr_built',
                    'bedrooms',
                    'bathrooms',
                ]
            ]
        )

    def filtrando_os_piores_imóveis(
        self,
        condition_max=2,
        yr_built_max=1980,
    ):

        # self.média_do_valor_do_ft2_por_cidade()

        df = self.df[
            (self.df['price'] / self.df['sqft_lot'] > self.df['price_of_ft2_per_city']) &
            (self.df['condition'] <= condition_max) &
            (self.df['yr_built'] < yr_built_max)
        ].sort_values(by='price').reset_index()

        self.piores_imóveis_filtrados = df

    def algoritmo_de_rankeamento_dos_piores_imóveis(self):
        # self.média_do_tamanho_do_terreno_por_cidade()
        # self.média_de_quantidade_quartos_por_cidade()
        # self.média_de_quantidade_banheiros_por_cidade()
        # self.filtrando_os_melhores_imóveis()

        self.rank_dos_piores_imóveis = {}
        cidades_mais_populosas = [
            'Seattle', 
            'Spokane',
            'Tacoma',
            'Vancouver',
            'Bellevue',
            'Kent',
            'Everett',
            'Renton',
            'Federal Way',
            'Spokane Valley',
        ]

        for id in self.piores_imóveis_filtrados['id']:
            self.rank_dos_piores_imóveis[id] = 0

            city = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['city'].iloc[0]

            price_of_ft2 = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['price_of_ft2'].iloc[0]

            price_of_ft2_per_city = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['price_of_ft2_per_city'].iloc[0]

            sqft_lot = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['sqft_lot'].iloc[0]

            plot_size_by_city = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['plot_size_by_city'].iloc[0]

            bedrooms = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['bedrooms'].iloc[0]

            bedrooms_by_city = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['bedrooms_by_city'].iloc[0]

            bathrooms = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['bathrooms'].iloc[0]

            bathrooms_by_city = self.piores_imóveis_filtrados[
                (self.piores_imóveis_filtrados['id'] == id)
            ]['bathrooms_by_city'].iloc[0]


            resultado = (price_of_ft2_per_city - price_of_ft2) / price_of_ft2 * 100


            if city not in cidades_mais_populosas:
                self.rank_dos_piores_imóveis[id] += 30

            if resultado < -50:
                self.rank_dos_piores_imóveis[id] += 30

            if sqft_lot < plot_size_by_city:
                self.rank_dos_piores_imóveis[id] += 20 

            if bedrooms < bedrooms_by_city:
                self.rank_dos_piores_imóveis[id] += 10

            if bathrooms < bathrooms_by_city:
                self.rank_dos_piores_imóveis[id] += 10

        self.rank_dos_piores_imóveis

    def os_5_piores_imóveis(self):

        notas_ordenadas = sorted(self.rank_dos_piores_imóveis.values(), reverse=True)
        cinco_maiores_notas = notas_ordenadas[:5]

        ids_cinco_maiores_notas = [k for k, v in self.rank_dos_piores_imóveis.items() if v in cinco_maiores_notas]
        self.os_piores_ids = ids_cinco_maiores_notas[:5]

        df = self.df[
            (self.df['id'].isin(self.os_piores_ids))
        ].sort_values(
            by='price',
            ascending=False
        ).reset_index()

        display(
            df[
                [
                    'id',
                    'city',
                    'price',
                    'sqft_lot',
                    'price_of_ft2',
                    'price_of_ft2_per_city',
                    'condition',
                    'yr_built',
                    'bedrooms',
                    'bathrooms',
                ]
            ]
        )

    def relatório(self):
        conteudo = f"""# Roof Imóveis:
## Overview:
Primeiro definimos o publico alvo, Casais com filhos, logo após isso, definimos as metricas, localização, preço, condição do imôvel, etc.

Com tudo isso em mente iniciamos as análises, e concluímos quais os 5 imóveis que ela deveria investir e o porquê, e, quais 5 você não recomendaria o investimento de jeito nenhum.

## 5 imóveis que deveria investir:
1. > O imóvel de ID: {self.ids[0]} está localizado em {self.df[(self.df['id'] == self.ids[0])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.ids[0])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.ids[0])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.ids[0])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.ids[0])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.ids[0])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.ids[0])]['sqft_living'].iloc[0]}.
2. > O imóvel de ID: {self.ids[1]} está localizado em {self.df[(self.df['id'] == self.ids[1])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.ids[1])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.ids[1])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.ids[1])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.ids[1])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.ids[1])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.ids[1])]['sqft_living'].iloc[0]}.
3. > O imóvel de ID: {self.ids[2]} está localizado em {self.df[(self.df['id'] == self.ids[2])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.ids[2])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.ids[2])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.ids[2])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.ids[2])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.ids[2])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.ids[2])]['sqft_living'].iloc[0]}.
4. > O imóvel de ID: {self.ids[3]} está localizado em {self.df[(self.df['id'] == self.ids[3])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.ids[3])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.ids[3])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.ids[3])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.ids[3])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.ids[3])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.ids[3])]['sqft_living'].iloc[0]}.
5. > O imóvel de ID: {self.ids[4]} está localizado em {self.df[(self.df['id'] == self.ids[4])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.ids[4])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.ids[4])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.ids[4])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.ids[4])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.ids[4])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.ids[4])]['sqft_living'].iloc[0]}.

## 5 imóveis que NÃO deveria investir:
1. > O imóvel de ID: {self.os_piores_ids[0]} está localizado em {self.df[(self.df['id'] == self.os_piores_ids[0])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.os_piores_ids[0])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.os_piores_ids[0])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.os_piores_ids[0])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.os_piores_ids[0])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.os_piores_ids[0])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.os_piores_ids[0])]['sqft_living'].iloc[0]}.
2. > O imóvel de ID: {self.os_piores_ids[1]} está localizado em {self.df[(self.df['id'] == self.os_piores_ids[1])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.os_piores_ids[1])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.os_piores_ids[1])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.os_piores_ids[1])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.os_piores_ids[1])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.os_piores_ids[1])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.os_piores_ids[1])]['sqft_living'].iloc[0]}.
3. > O imóvel de ID: {self.os_piores_ids[2]} está localizado em {self.df[(self.df['id'] == self.os_piores_ids[2])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.os_piores_ids[2])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.os_piores_ids[2])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.os_piores_ids[2])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.os_piores_ids[2])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.os_piores_ids[2])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.os_piores_ids[2])]['sqft_living'].iloc[0]}.
4. > O imóvel de ID: {self.os_piores_ids[3]} está localizado em {self.df[(self.df['id'] == self.os_piores_ids[3])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.os_piores_ids[3])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.os_piores_ids[3])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.os_piores_ids[3])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.os_piores_ids[3])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.os_piores_ids[3])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.os_piores_ids[3])]['sqft_living'].iloc[0]}.
5. > O imóvel de ID: {self.os_piores_ids[4]} está localizado em {self.df[(self.df['id'] == self.os_piores_ids[4])]['city'].iloc[0]}, o mesmo está sendo vendido por {self.df[(self.df['id'] == self.os_piores_ids[4])]['price'].iloc[0]} é o valor do pé quadrado é {self.df[(self.df['id'] == self.os_piores_ids[4])]['price_of_ft2'].iloc[0]}, o tamanho do terreno é {self.df[(self.df['id'] == self.os_piores_ids[4])]['sqft_lot'].iloc[0]}, ele tem {self.df[(self.df['id'] == self.os_piores_ids[4])]['bedrooms'].iloc[0]} quarto(s) e {self.df[(self.df['id'] == self.os_piores_ids[4])]['bathrooms'].iloc[0]} banheiro(s), por fim sua área habitável é de {self.df[(self.df['id'] == self.os_piores_ids[4])]['sqft_living'].iloc[0]}.

## Observações:
Primeiro, para obtermos uma análise mais precisa precisamos saber qual o objetivo da compra do imóvel, revenda? Aluguel? Quem é o público alvo? Casais sem filhos? Casais com filhos? Solteiros? Ou Estudantes. Que tipo de imóvel estamos buscando? De alto padrão? Populares? Qual o valor máximo do investimento?

Agora em relação aos dados, seria mais simples se tivessímos acesso ao valor do pé quadrado por cidade, em relação aos imóveis recomendados, antes de efetuamos a compra é preciso observar se eles são bem localizado, por exemplo, escolas próximas, e se é próximo ao centro da cidade, etc.

## Possivel Publico alvo:

1. Casais sem filhos.
2. Casais com filhos.
4. Solteiros.
5. Estudantes.

## Metricas de avaliação do imôvel:

1. Localização, o imôvel deve está em uma das 10 cidades mais populosas de Washington.

2. Valor do ft2 (pé quadrado), deve ser menor que a média da cidade.

3. Preço, deve ser menor que a média da cidade.

4. Condição do imôvel, a condição do deve ser 5.

5. Tamanho do terreno, deve ser maior que a média da cidade

6. Quantidade de quartos, deve ser maior que a média da cidade.

7. Quantidade de banheiros, deve ser maior que a média da cidade.

8. Ano de construção, deve ser maior que 1980.

## Insights técnicos da análise:

1. Quando comecei a analisar os dados me deparei com a coluna `bathrooms`, nela observei que avia informações de casas com, por exemplo, 2.25, 4.50, 1.75 banheiros, o que não faz sentindo para a minha análise, então optei por convertê-las em inteiros.

2. Logo após me deparar com a situação da coluna `bathrooms`, encontrei algo semelhante na coluna `floors`, por exemplo, casas com 1.5, 2.5, 3.5 andares, na coluna `floors` encontrando o número de andares de uma casa, portanto, optei por convertê-las em inteiros.

---
Linkedin: <https://www.linkedin.com/in/samuel-barbosa-dev/> 

E-mail: <samueloficial@protonmail.com>
"""
        with open(f"README.md", "w") as file:
                file.write(str(conteudo))


if __name__ == '__main__':
    roof_imóveis = RoofImóveis()

In [642]:
roof_imóveis.média_do_valor_do_ft2_por_cidade()

/tmp/ipykernel_10688/1407862707.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['price_of_ft2'] = df['price_of_ft2_per_city'] = df['price'] / df['sqft_lot']


In [ ]:
roof_imóveis.média_do_tamanho_do_terreno_por_cidade()

In [ ]:
roof_imóveis.média_de_quantidade_quartos_por_cidade()

In [ ]:
roof_imóveis.média_de_quantidade_banheiros_por_cidade()

In [ ]:
roof_imóveis.filtrando_os_melhores_imóveis()

In [ ]:
roof_imóveis.algoritmo_de_rankeamento_de_imóveis()

In [ ]:
roof_imóveis.os_5_melhores_imóveis()

,id,city,price,sqft_lot,price_of_ft2,price_of_ft2_per_city,condition,yr_built,bedrooms,bathrooms
0,2144800146,Seattle,257500.0,9334,27.587315,92.800000,5,1981,3,2
1,3348401382,Seattle,318000.0,12662,25.114516,92.800000,5,1982,3,2
2,4022902715,Seattle,525000.0,10277,51.084947,92.800000,5,1993,5,3
3,5067400032,Seattle,550000.0,14400,38.194444,92.800000,5,1985,3,2
4,9485950310,Kent,610000.0,37058,16.460683,33.771107,5,1984,4,3


In [ ]:
roof_imóveis.filtrando_os_piores_imóveis()

In [ ]:
roof_imóveis.algoritmo_de_rankeamento_dos_piores_imóveis()

In [ ]:
roof_imóveis.os_5_piores_imóveis()

,id,city,price,sqft_lot,price_of_ft2,price_of_ft2_per_city,condition,yr_built,bedrooms,bathrooms
0,2767604170,Seattle,975000.0,5000,195.000000,92.800000,2,1900,3,3
1,3885808035,Kirkland,619500.0,5202,119.088812,60.305302,2,1911,6,1
2,9407110710,North Bend,322000.0,8400,38.333333,25.262532,2,1979,3,1
3,1771100330,Woodinville,250000.0,10384,24.075501,22.672789,2,1976,3,2
4,3523029059,Vashon,181000.0,10807,16.748404,10.452218,2,1949,2,1


In [ ]:
roof_imóveis.relatório()